In [ ]:
import random
import numpy as np

def get_datafile_input(datafile_input):
    with open(datafile_input, 'r') as input_data:
        usc_places = int(input_data.readline().strip())
        usc_points = []
        for _ in range(usc_places):
            place = tuple(map(int, input_data.readline().strip().split()))
            usc_points.append(place)
        return usc_points

def save_output_file(output_filename, cummulative_traveldist, best_way, usc_points):
    with open(output_filename, 'w') as generated_output:
        generated_output.write(f"{cummulative_traveldist}\n")
        for point_index in best_way:
            x, y, z = usc_points[point_index]
            generated_output.write(f"{x} {y} {z}\n")
        x, y, z = usc_points[best_way[0]]
        generated_output.write(f"{x} {y} {z}\n")

def distance_measure(usc_p1, usc_p2):
    return np.sqrt(np.sum((np.array(usc_p1) - np.array(usc_p2))**2))

def calculate_dist_matrix(usc_points):
    usc_places = len(usc_points)
    dist_matrix = np.zeros((usc_places, usc_places))
    for i in range(usc_places):
        for j in range(i + 1, usc_places):
            dist = distance_measure(usc_points[i], usc_points[j])
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist
    return dist_matrix

def calculate_total_dist(way, dist_matrix):
    total_distance = 0
    for i in range(len(way) - 1):
        p1 = way[i]
        p2 = way[i + 1]
        total_distance += dist_matrix[p1, p2]
    total_distance += dist_matrix[way[-1], way[0]]
    return total_distance

def tournament_selection(population, fitness_values, size):
    parents_selected = []
    while len(parents_selected) < len(population):
        indices = random.sample(range(len(population)), size)
        fitness = [fitness_values[i] for i in indices]
        winner_index = indices[fitness.index(min(fitness))]
        parents_selected.append(population[winner_index])
    return parents_selected

def crossover_ordered(p1, p2):
    crossover_start = random.randint(0, len(p1) - 1)
    crossover_end = random.randint(crossover_start, len(p1) - 1)
    child = [-1] * len(p1)
    child[crossover_start:crossover_end] = p1[crossover_start:crossover_end]
    j = crossover_end
    for i in range(len(p1)):
        if p2[i] not in child:
            while child[j] != -1:
                j = (j + 1) % len(p1)
            child[j] = p2[i]
    return child

def mutate_swap(way, rate_of_mutation):
    if random.random() < rate_of_mutation:
        i, j = random.sample(range(len(way)), 2)
        way[i], way[j] = way[j], way[i]
    return way

def NN_initialization(usc_points, dist_matrix):
    way = [random.randint(0, usc_points - 1)]
    while len(way) < usc_points:
        current_point = way[-1]
        nearest_points = [point for point in range(usc_points) if point not in way]
        nearest_points.sort(key=lambda point: dist_matrix[current_point, point])
        next_point = nearest_points[0]
        way.append(next_point)
    return way

def dynamic_rate_of_mutation(current_generation, max_generations):
    return max(0.1, 0.2 - (0.1 / max_generations) * current_generation)

if __name__ == "__main__":
    datafile_input = "input.txt"
    output_filename = "output.txt"

    usc_points = get_datafile_input(datafile_input)
    city_points = len(usc_points)
    population_size = 100
    generations = 200
    tournament_size = 20

    dist_matrix = calculate_dist_matrix(usc_points)

    population = [NN_initialization(city_points, dist_matrix) for _ in range(population_size)]

    for generation in range(generations):
        new_population = []

        fitness_values = [calculate_total_dist(way, dist_matrix) for way in population]

        elite_index = fitness_values.index(min(fitness_values))
        elite_way = population[elite_index]

        parents = tournament_selection(population, fitness_values, tournament_size)

        rate_of_mutation = dynamic_rate_of_mutation(generation, generations)

        for i in range(0, len(parents), 2):
            parent1 = parents[i]
            parent2 = parents[i + 1]

            child1 = crossover_ordered(parent1, parent2)
            child2 = crossover_ordered(parent2, parent1)

            child1 = mutate_swap(child1, rate_of_mutation)
            child2 = mutate_swap(child2, rate_of_mutation)

            new_population.extend([child1, child2])

        worst_index = fitness_values.index(max(fitness_values))
        new_population[worst_index] = elite_way

        population = new_population

    best_way = min(population, key=lambda way: calculate_total_dist(way, dist_matrix))
    best_distance = calculate_total_dist(best_way, dist_matrix)

    save_output_file(output_filename, best_distance, best_way, usc_points)
